<a href="https://colab.research.google.com/github/kemwe/LLM/blob/master/implemenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nvcc4jupyter
!pip install --upgrade diffusers
!pip install transformers accelerate safetensors

%load_ext nvcc4jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.3
    Uninstalling diffusers-0.30.3:
      Successfully uninstalled diffusers-0.30.3
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpz32rbrm7".


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



In [4]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}") # Print the device being used

Using device: cuda


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
messages = [
   {"role": "user", "content": "How do chat templates work?"},
   {"role": "assistant", "content": "Chat templates help  LLMs like me generate more coherent responses by providing a structured way to organize the conversation."},
   {"role": "user", "content": "How do I use them?"},
]

In [9]:
model_inputs = tokenizer.apply_chat_template(messages,add_generation_prompt = True, return_tensors='pt')
model_inputs

tensor([[  529, 29989,  1792, 29989, 29958,    13,  5328,   437, 13563, 17475,
           664, 29973,     2, 29871,    13, 29966, 29989,   465, 22137, 29989,
         29958,    13,  1451,   271, 17475,  1371, 29871,   365, 26369, 29879,
           763,   592,  5706,   901, 16165,   261,   296, 20890,   491, 13138,
           263,  2281,  2955,   982,   304,  2894,   675,   278, 14983, 29889,
             2, 29871,    13, 29966, 29989,  1792, 29989, 29958,    13,  5328,
           437,   306,   671,   963, 29973,     2, 29871,    13, 29966, 29989,
           465, 22137, 29989, 29958,    13]])

In [11]:
generated_ids = model.generate(model_inputs.to(device))
generated_ids

tensor([[  529, 29989,  1792, 29989, 29958,    13,  5328,   437, 13563, 17475,
           664, 29973,     2, 29871,    13, 29966, 29989,   465, 22137, 29989,
         29958,    13,  1451,   271, 17475,  1371, 29871,   365, 26369, 29879,
           763,   592,  5706,   901, 16165,   261,   296, 20890,   491, 13138,
           263,  2281,  2955,   982,   304,  2894,   675,   278, 14983, 29889,
             2, 29871,    13, 29966, 29989,  1792, 29989, 29958,    13,  5328,
           437,   306,   671,   963, 29973,     2, 29871,    13, 29966, 29989,
           465, 22137, 29989, 29958,    13,  1762,   671, 13563, 17475, 29892,
           366,   508,  1101,  1438,  6576, 29901,    13,    13, 29896, 29889,
          4522,   297,   304,   596,   365, 26369, 13563,  7451,  7481, 29889,
            13, 29906, 29889, 16297,   373,   278,   376,  5776,  9884, 29908,
          4434,   297,   278,  2175, 29899,  3179,  6143, 29889,    13, 29941,
         29889, 16297,   373,   278,   376,  2528, 2

In [14]:
decoded_response = tokenizer.batch_decode(generated_ids)
print(decoded_response[0])

<|user|>
How do chat templates work?</s> 
<|assistant|>
Chat templates help  LLMs like me generate more coherent responses by providing a structured way to organize the conversation.</s> 
<|user|>
How do I use them?</s> 
<|assistant|>
To use chat templates, you can follow these steps:

1. Log in to your LLM chatbot platform.
2. Click on the "Templates" tab in the left-hand menu.
3. Click on the "Add Template" button.
4. Choose the template you want to use from the list of available templates.
5. Customize the template by adding your own text, images, and other elements.
6. Save your template and assign it to a conversation.

Once you have created a chat template, you can use it to generate responses to common customer queries or to customize responses based on the conversation.</s>


In [24]:
generated_ids = model.generate(model_inputs.to(device),max_new_tokens=128, do_sample=True,temperature=0.5,top_p=0.5)
generated_ids

tensor([[  529, 29989,  1792, 29989, 29958,    13,  5328,   437, 13563, 17475,
           664, 29973,     2, 29871,    13, 29966, 29989,   465, 22137, 29989,
         29958,    13,  1451,   271, 17475,  1371, 29871,   365, 26369, 29879,
           763,   592,  5706,   901, 16165,   261,   296, 20890,   491, 13138,
           263,  2281,  2955,   982,   304,  2894,   675,   278, 14983, 29889,
             2, 29871,    13, 29966, 29989,  1792, 29989, 29958,    13,  5328,
           437,   306,   671,   963, 29973,     2, 29871,    13, 29966, 29989,
           465, 22137, 29989, 29958,    13,  1762,   671, 13563, 17475, 29892,
           366,   508,  1101,  1438,  6576, 29901,    13,    13, 29896, 29889,
          4522,   297,   304,   596,   365, 26369, 13563,  7451,  7481, 29889,
            13, 29906, 29889, 16297,   373,   278,   376,  5776,  9884, 29908,
          4434,   297,   278,  2175, 29899,  3179,  6143, 29889,    13, 29941,
         29889, 16297,   373,   278,   376,  4373, 2

In [26]:
decoded_response = tokenizer.batch_decode(generated_ids)
print(decoded_response[0])

<|user|>
How do chat templates work?</s> 
<|assistant|>
Chat templates help  LLMs like me generate more coherent responses by providing a structured way to organize the conversation.</s> 
<|user|>
How do I use them?</s> 
<|assistant|>
To use chat templates, you can follow these steps:

1. Log in to your LLM chatbot platform.
2. Click on the "Templates" tab in the left-hand menu.
3. Click on the "New Template" button.
4. Choose the type of chat template you want to create. For example, you can create a "Welcome" template, a "Help" template, or a "Say Hello" template.
5. Customize the template by adding your own text and formatting.
6. Save your template and assign it to a specific conversation.

